In [ ]:
ruta_arbol = "data/resultados/dataset_arbol.parquet" 

In [ ]:
from pyspark.sql import SparkSession

# Detener sesión anterior
try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("Preparación dataset para entrenamiento del modelo predictivo") \
    .config("spark.master", "local[*]") \
    .config("spark.driver.memory", "20g") \
    .config("spark.executor.memory", "20g") \
    .getOrCreate()

In [ ]:
df_arbol = spark.read.parquet(ruta_arbol)

# Análisis exploratorio

In [ ]:
# Mostrar el esquema del dataset
df_arbol.printSchema()

In [ ]:
# Contar el número total de registros
print(f"Número total de ingresos hospitalarios: {df_arbol.count()}")

In [ ]:
import pandas as pd

# Obtener el resumen del DataFrame de Spark y convertirlo en Pandas
df_stats = df_arbol.describe().toPandas()

df_stats

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Convertir el DataFrame de Spark a Pandas
df_pandas = df_arbol.select(
    "sexo", "estado_civil", "tipo_seguro", "grupo_poblacional", "edad_categoria", "muerte_durante_ingreso"
).toPandas()

# Configuración del estilo
sns.set_theme(style="whitegrid")  # Fondo blanco con grid suave
palette = sns.color_palette("Set2")  # Paleta de colores moderna y profesional

# Crear la figura y los subgráficos
fig, axes = plt.subplots(3, 2, figsize=(14, 12))
axes = axes.flatten()

# Variables categóricas a graficar
categorical_cols = [
    "sexo", "estado_civil", "tipo_seguro", "grupo_poblacional", "edad_categoria", "muerte_durante_ingreso"
]

# Crear un gráfico de barras para cada variable categórica
for i, col in enumerate(categorical_cols):
    ax = axes[i]
    sns.countplot(
        y=df_pandas[col],
        ax=ax,
        order=df_pandas[col].value_counts().index,
        palette=palette  # Aplicamos la paleta de colores
    )
    ax.set_title(f"Distribución de {col}", fontsize=14, fontweight="bold", color="#333")
    ax.set_xlabel("Cantidad", fontsize=12, color="#555")
    ax.set_ylabel(col, fontsize=12, color="#555")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

# Ajustar el espaciado para mejorar la legibilidad
plt.tight_layout()

# Mostrar la figura
plt.show()

In [ ]:
from pyspark.sql.functions import explode, count

# Convertir la lista de dominios en filas individuales y contar frecuencia
df_dominio_frecuencia = df_arbol.select(explode(df_arbol["dominios"]).alias("dominio")) \
    .groupBy("dominio") \
    .agg(count("*").alias("frecuencia")) \
    .orderBy("frecuencia", ascending=False)  # Aquí corregimos el orden

# Convertir a Pandas para visualización
df_dominio_frecuencia_pandas = df_dominio_frecuencia.toPandas()

# Mostrar los 10 dominios más frecuentes
df_dominio_frecuencia_pandas.head(10)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Seleccionar los 20 dominios más frecuentes
top_n = 20
df_top_dominio = df_dominio_frecuencia_pandas.head(top_n)

# Crear el gráfico de barras
plt.figure(figsize=(12, 6))
sns.barplot(y=df_top_dominio["dominio"], x=df_top_dominio["frecuencia"], palette="viridis")
plt.xlabel("Frecuencia")
plt.ylabel("Dominio ICD")
plt.title(f"Top {top_n} dominios de diagnóstico más frecuentes")
plt.show()

In [ ]:
from pyspark.sql.functions import explode, count

# Explode para convertir la lista de dominios en filas individuales
df_dominio_edad = df_arbol.select(explode(df_arbol["dominios"]).alias("dominio"), "edad_categoria")

# Contar la frecuencia de cada diagnóstico
df_dominio_frecuencia = df_dominio_edad.groupBy("dominio").agg(count("*").alias("frecuencia"))

# Seleccionar los 10 diagnósticos más frecuentes
top_diagnosticos = df_dominio_frecuencia.orderBy("frecuencia", ascending=False).limit(10)

# Filtrar el dataset para incluir solo estos diagnósticos
df_top_dominio_edad = df_dominio_edad.join(top_diagnosticos, on="dominio", how="inner")

# Contar la cantidad de cada diagnóstico por grupo de edad
df_dominio_edad_frecuencia = df_top_dominio_edad.groupBy("dominio", "edad_categoria") \
    .agg(count("*").alias("frecuencia")) \
    .orderBy("dominio", "edad_categoria")

# Convertir a Pandas para visualización
df_dominio_edad_pandas = df_dominio_edad_frecuencia.toPandas()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración del gráfico
plt.figure(figsize=(14, 6))
sns.barplot(data=df_dominio_edad_pandas, x="dominio", y="frecuencia", hue="edad_categoria", palette="coolwarm")

# Etiquetas y título
plt.xlabel("Diagnóstico")
plt.ylabel("Frecuencia")
plt.title("Top 10 Diagnósticos más Frecuentes por Grupo de Edad")
plt.xticks(rotation=45, ha="right")  # Rotar etiquetas para mejor lectura
plt.legend(title="Grupo de Edad")

# Mostrar gráfico
plt.show()

In [ ]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

# Contar la frecuencia de cada diagnóstico por grupo de edad
df_dominio_edad_frecuencia = df_dominio_edad.groupBy("edad_categoria", "dominio") \
    .agg(count("*").alias("frecuencia"))

# Crear una ventana de particionamiento por grupo de edad y ordenar por frecuencia descendente
window_spec = Window.partitionBy("edad_categoria").orderBy(df_dominio_edad_frecuencia["frecuencia"].desc())

# Añadir una columna con el ranking de diagnósticos dentro de cada grupo de edad
df_dominio_edad_ranked = df_dominio_edad_frecuencia.withColumn("rank", row_number().over(window_spec))

# Filtrar solo los 5 diagnósticos más frecuentes por grupo de edad
df_top5_por_edad = df_dominio_edad_ranked.filter("rank <= 5").orderBy("edad_categoria", "rank")

# Convertir a Pandas para visualización
df_top5_por_edad_pandas = df_top5_por_edad.toPandas()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración del gráfico
plt.figure(figsize=(14, 8))
sns.barplot(data=df_top5_por_edad_pandas, x="edad_categoria", y="frecuencia", hue="dominio", palette="viridis")

# Etiquetas y título
plt.xlabel("Grupo de Edad")
plt.ylabel("Frecuencia")
plt.title("Top 5 Diagnósticos más Frecuentes por Grupo de Edad")
plt.xticks(rotation=45)
plt.legend(title="Diagnóstico", bbox_to_anchor=(1.05, 1), loc="upper left")

# Mostrar gráfico
plt.show()

In [ ]:
import numpy as np

# Calcular estadísticas básicas de la frecuencia de dominios
mean_freq = df_dominio_frecuencia_pandas["frecuencia"].mean()
std_freq = df_dominio_frecuencia_pandas["frecuencia"].std()

# Definir un umbral de rareza (menor al 5% del promedio)
umbral_raro = mean_freq * 0.05

# Filtrar dominios raros
dominios_raros = df_dominio_frecuencia_pandas[df_dominio_frecuencia_pandas["frecuencia"] < umbral_raro]

# Mostrar dominios raros
print(f"Se encontraron {len(dominios_raros)} dominios con frecuencia menor a {umbral_raro:.2f}")
dominios_raros

In [ ]:
# Contar el número total de dominios únicos
num_dominios_totales = df_arbol.select(explode(df_arbol["dominios"]).alias("dominio")) \
    .select("dominio").distinct().count()

print(f"Número total de dominios únicos en el dataset: {num_dominios_totales}")